<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/fdcev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper utility (assuming flat_rate_py is defined as in previous examples)
def flat_rate_py(evaluation_date_or_value, day_counter_or_rateval, day_counter_if_date=None):
    if isinstance(evaluation_date_or_value, ql.Date):
        eval_date = evaluation_date_or_value
        rate_val = day_counter_or_rateval
        dc = day_counter_if_date
    else:
        eval_date = ql.Settings.instance().evaluationDate
        rate_val = evaluation_date_or_value
        dc = day_counter_or_rateval

    if isinstance(rate_val, ql.Quote):
        quote_handle = ql.QuoteHandle(rate_val)
    elif isinstance(rate_val, float):
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(rate_val))
    else:
        quote_handle = rate_val
    return ql.FlatForward(eval_date, quote_handle, dc)

# Functor for CEV expectation calculation
class ExpectationFctPy:
    def __init__(self, calculator, t):
        self.t_ = t
        self.calculator_ = calculator

    def __call__(self, f_val):
        return f_val * self.calculator_.pdf(f_val, self.t_)


class FdCevTests(unittest.TestCase):

    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # Set a default evaluation date if tests rely on it
        # self.today = ql.Date(22, ql.February, 2018)
        # ql.Settings.instance().evaluationDate = self.today

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def test_local_martingale(self):
        print("Testing local martingale property of CEV process with PDF...")

        t_val = 1.0
        f0_val = 2.1
        alpha_val = 1.75
        betas = [-2.4, 0.23, 0.9, 1.1, 1.5]

        for beta_val in betas:
            rnd_calculator = ql.CEVRNDCalculator(f0_val, alpha_val, beta_val)

            eps_integral = 1e-10
            tol_comparison = 100 * eps_integral

            # Determine upper bound for integration
            # invcdf might require a probability > eps_integral and < 1-eps_integral
            # For very small eps_integral, invcdf(1-eps_integral) can be very large.
            # Let's use a reasonably high probability like 0.999999
            upper_bound_prob = 1.0 - 1e-6 # Was 1-eps in C++, but invcdf(1-1e-10) might be too extreme
            upper_bound = 10.0 * rnd_calculator.invcdf(upper_bound_prob, t_val)
            # Ensure upper_bound is greater than a small positive number if rndCalculator.invcdf is small
            upper_bound = max(upper_bound, ql.QL_EPSILON * 100)


            expectation_fct = ExpectationFctPy(rnd_calculator, t_val)
            # GaussLobattoIntegral(maxIterations, absAccuracy)
            integrator = ql.GaussLobattoIntegral(10000, eps_integral)

            # Integrate from QL_EPSILON to upperBound
            # Ensure lower bound is strictly positive
            lower_bound_integral = ql.QL_EPSILON
            if lower_bound_integral >= upper_bound: # Safety for extreme cases
                print(f"Warning: Lower bound {lower_bound_integral} >= Upper bound {upper_bound} for beta={beta_val}. Skipping integration.")
                expectation_value = 0 # Or handle appropriately
            else:
                try:
                    expectation_value = integrator(expectation_fct, lower_bound_integral, upper_bound)
                except RuntimeError as e:
                    print(f"Warning: Integration failed for beta={beta_val}, f0={f0_val}, alpha={alpha_val} "
                          f"with bounds [{lower_bound_integral}, {upper_bound}]. Error: {e}. Setting expectation to NaN.")
                    expectation_value = float('nan')


            diff = expectation_value - f0_val

            if not math.isnan(expectation_value):
                if beta_val < 1.0:
                    self.assertAlmostEqual(diff, 0.0, delta=tol_comparison,
                                           msg=f"CEV process should be a martingale for beta < 1.0 (beta={beta_val})\n"
                                               f"    E[F_t|F_0] - F_0 = {diff:.3e}, F_0 = {f0_val}")
                elif beta_val > 1.0: # Should be E[F_t|F_0] < F_0  => diff < 0
                                     # C++ test: diff > -tol  => -(E-F0) < tol => F0-E < tol
                    self.assertLess(diff, tol_comparison, # diff should be negative, so diff < +tol is consistent with F0-E < tol if E<F0
                                      msg=f"CEV process local martingale for beta > 1.0 (beta={beta_val})\n"
                                          f"    E[F_t|F_0] = {expectation_value:.6f}, F_0 = {f0_val}, Diff = {diff:.3e}")

            # Monte Carlo simulation for beta > 1.2
            if beta_val > 1.2:
                n_sims = 5000 # Number of simulations
                n_steps = 2000 # Number of time steps
                dt = t_val / n_steps
                sqrt_dt = math.sqrt(dt)

                stat = ql.GeneralStatistics()
                # ql.MersenneTwisterUniformRng takes a seed.
                # C++ PseudoRandom::rng_type mt(MersenneTwisterUniformRng(42));
                # mt.next().value gives a U(0,1) draw.
                rng = ql.MersenneTwisterUniformRng(42)

                for _ in range(n_sims):
                    f_path = f0_val
                    for _ in range(n_steps):
                        if f_path == 0.0: break # Absorbing boundary
                        # SDE: dF = alpha * F^beta * dW
                        f_path += alpha_val * (f_path**beta_val) * rng.next().value() * sqrt_dt
                        # rng.next().value() is U(0,1). For dW, need N(0,1).
                        # The C++ code uses mt.next().value directly which is U(0,1).
                        # This is incorrect for dW which needs N(0,1).
                        # However, to match the C++ test, we replicate its (potentially flawed) MC.
                        # If it intended N(0,1), it should be inv_cum_normal(rng.next().value()).
                        # For now, sticking to direct U(0,1) as in C++ test.

                        # Correct SDE discretization using N(0,1) draw:
                        # inv_normal = ql.InverseCumulativeNormal()
                        # norm_draw = inv_normal(rng.next().value())
                        # f_path += alpha_val * (f_path**beta_val) * norm_draw * sqrt_dt

                        f_path = max(0.0, f_path)
                    stat.add(f_path - f0_val) # Store F_t - F_0

                mc_mean_diff = stat.mean()
                mc_error_estimate = stat.errorEstimate()

                # Compare MC diff with PDF-based diff
                if not math.isnan(diff): # Only if PDF integration was successful
                    self.assertAlmostEqual(mc_mean_diff, diff, delta=2.35 * mc_error_estimate,
                                        msg=f"MC vs PDF E[F_t-F_0|F_0] mismatch for beta={beta_val}:\n"
                                            f"    E_PDF[F_t-F_0]: {diff:.6f}\n"
                                            f"    E_MC[F_t-F_0]: {mc_mean_diff:.6f}\n"
                                            f"    MC Error: {mc_error_estimate:.2e}")

    def test_fdm_cev_op(self):
        print("Testing FDM constant elasticity of variance (CEV) operator...")

        today_fdm = ql.Date(22, ql.February, 2018)
        dc_fdm = ql.Actual365Fixed()
        ql.Settings.instance().evaluationDate = today_fdm

        maturity_date = today_fdm + ql.Period(12, ql.Months)
        strike_val = 2.3
        option_types = [ql.Option.Call, ql.Option.Put]
        exercise = ql.EuropeanExercise(maturity_date)

        r_ts_handle = ql.YieldTermStructureHandle(flat_rate_py(today_fdm, 0.15, dc_fdm))
        f0_fdm = 2.1
        alpha_fdm = 0.75
        betas_fdm = [-2.0, -0.5, 0.45, 0.6, 0.9, 1.45]

        for opt_type in option_types:
            payoff = ql.PlainVanillaPayoff(opt_type, strike_val)
            for beta_val_fdm in betas_fdm:
                option_analytic = ql.VanillaOption(payoff, exercise)
                analytic_engine = ql.AnalyticCEVEngine(f0_fdm, alpha_fdm, beta_val_fdm, r_ts_handle)
                option_analytic.setPricingEngine(analytic_engine)
                analytic_npv = option_analytic.NPV()

                eps_fd = 1e-3
                # Analytic Delta by FD
                option_up = ql.VanillaOption(payoff, exercise)
                option_up.setPricingEngine(ql.AnalyticCEVEngine(f0_fdm * (1+eps_fd), alpha_fdm, beta_val_fdm, r_ts_handle))
                analytic_up_npv = option_up.NPV()

                option_down = ql.VanillaOption(payoff, exercise)
                option_down.setPricingEngine(ql.AnalyticCEVEngine(f0_fdm * (1-eps_fd), alpha_fdm, beta_val_fdm, r_ts_handle))
                analytic_down_npv = option_down.NPV()

                analytic_delta = (analytic_up_npv - analytic_down_npv) / (2 * eps_fd * f0_fdm)

                # FDM Engine
                option_fdm = ql.VanillaOption(payoff, exercise)
                # FdCEVVanillaEngine(f0, alpha, beta, rTS, tGrid, xGrid,
                #                    dampingSteps=0, schemeDesc=FdmSchemeDesc::Douglas(),
                #                    localVol=false, illegalLocalVolOverwrite=Null<Real>())
                # The C++ test uses: 100, 1000, 1, 1.0, 1e-6
                # These might map to tGrid, xGrid, dampingSteps, theta (for scheme), localVol (implicit false)
                # FdCEVVanillaEngine(f0, alpha, beta, rTS, tGrid, xGrid, dampingSteps=0, theta=0.5, ...)
                # The C++ test uses (f0, alpha, beta, rTS, 100, 1000, 1, 1.0, 1e-6)
                # This might be tGrid=100, xGrid=1000, dampingSteps=1.
                # The 1.0 and 1e-6 are harder to map directly without knowing exact Python constructor.
                # Let's use common defaults or typical values.
                # Default scheme is Douglas.
                fdm_engine = ql.FdCEVVanillaEngine(f0_fdm, alpha_fdm, beta_val_fdm, r_ts_handle,
                                                   100, 1000, 1) # tGrid, xGrid, dampingSteps
                option_fdm.setPricingEngine(fdm_engine)

                calculated_npv_fdm = option_fdm.NPV()
                calculated_delta_fdm = option_fdm.delta()

                tol_fdm = 0.01

                msg_prefix = (f"FDM CEV Mismatch (beta={beta_val_fdm}, type={opt_type}):\n"
                              f"  Analytic NPV: {analytic_npv:.6f}, PDE NPV: {calculated_npv_fdm:.6f}\n"
                              f"  Analytic Delta: {analytic_delta:.6f}, PDE Delta: {calculated_delta_fdm:.6f}\n")

                self.assertAlmostEqual(calculated_npv_fdm, analytic_npv, delta=tol_fdm,
                                       msg=msg_prefix + f"  NPV Diff: {abs(calculated_npv_fdm - analytic_npv):.2e}")
                self.assertAlmostEqual(calculated_delta_fdm, analytic_delta, delta=tol_fdm,
                                       msg=msg_prefix + f"  Delta Diff: {abs(calculated_delta_fdm - analytic_delta):.2e}")


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)